In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('../input/data/train_dataset/train_data.csv')
test_df = pd.read_csv('../input/data/train_dataset/test_data.csv')

### new_userID

In [3]:
# UserID를 시험지로 나눔
arr = []
for ele in train_df.assessmentItemID.values:
    arr.append(ele[:3])
newID = [str(e1) + str(e2) for e1, e2 in zip(arr, train_df["userID"])]
train_df["newUserID"] = newID

In [11]:
train_df.groupby('newUserID').count()['userID'].describe()

count    17752.000000
mean       127.680599
std        157.643338
min          3.000000
25%         20.000000
50%         62.000000
75%        181.000000
max       1269.000000
Name: userID, dtype: float64

In [15]:
train_df.groupby('newUserID').count()['userID'].quantile(0.85)

273.35000000000036

In [22]:
tgt_arr = test_df[test_df['answerCode'] == -1]['assessmentItemID'].sort_values().unique()

In [30]:
test_df[(test_df['answerCode'] == -1 )]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
1035,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289
1706,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080
3023,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660
4283,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611
4670,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422
...,...,...,...,...,...,...
260052,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615
260067,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636
260082,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402


In [34]:
tmp = test_df[test_df['answerCode'] != -1]

In [39]:
print(len(train_df)+len(test_df)-744)

2525956


In [36]:
total = pd.concat([train_df,tmp])
total

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244


In [3]:
pseudo_df = pd.read_csv('/opt/ml/code/output/LGBM 31개 features 1000 boost.csv')['prediction']

In [4]:
pseudo_labels = np.where(pseudo_df >= 0.5, 1, 0)

In [7]:
test_df_ = test_df.copy()
test_df_

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623
...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


In [30]:
test_df_.loc[test_df_.answerCode ==-1, 'answerCode'] = pseudo_labels

In [31]:
test_df_['answerCode'].unique()

array([1, 0])

In [32]:
pseudo_labels[-6:]

array([1, 0, 1, 1, 1, 1])

In [33]:
test_df[test_df['answerCode'] == -1].tail()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260052,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615
260067,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636
260082,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402
260113,7439,A040130005,A040000130,-1,2020-10-14 23:10:03,8832


In [34]:
test_df_[test_df_['userID']==7395]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260029,7395,A030076001,A030000076,0,2020-07-10 06:36:14,475
260030,7395,A030076002,A030000076,1,2020-07-10 06:36:41,475
260031,7395,A030076003,A030000076,1,2020-07-10 06:43:01,475
260032,7395,A030076004,A030000076,1,2020-07-10 06:44:00,475
260033,7395,A030076005,A030000076,1,2020-07-10 06:44:55,475
260034,7395,A070070001,A070000070,0,2020-07-30 03:25:05,6801
260035,7395,A070070002,A070000070,0,2020-07-30 03:25:09,6801
260036,7395,A070070003,A070000070,0,2020-07-30 03:25:11,6801
260037,7395,A070070004,A070000070,1,2020-07-30 03:25:19,6801
260038,7395,A070070005,A070000070,0,2020-07-30 03:25:27,6801
